## Downlaod Data From GEE

In [1]:
import ee
ee.Authenticate()

Enter verification code:  4/1AX4XfWjbYn4NCjCwClolurtnA9vovbz7iLsDmAdUtpXFthq8bYa_xOKJxVo



Successfully saved authorization token.


In [2]:
ee.Initialize()

In [27]:
from download_utils import Mapdisplay
import folium
import geehydro
import time 

In [97]:
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1')
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017");
Country = countries.filter(ee.Filter.eq("country_na","China"));

landsat = l8.filterDate('2019-01-01', '2019-10-31') \
            .filterBounds(Country) 

## this is a cloud masking method in EE
composite = ee.Algorithms.Landsat.simpleComposite(**{
  'collection': landsat,
  'asFloat': True
})
## add layer info
rgbVis = {'bands': ["B4", "B3", "B2"], 'min':0, 'max': 0.3}

In [98]:
Map = folium.Map(location=[40, -100], zoom_start=4)
#Map.setOptions('SATELLITE')
Map.addLayer(composite, rgbVis, "RGB")
Map.centerObject(Country, 4)
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map

In [99]:
projection = composite.select('B2').projection().getInfo();

In [100]:
projection

{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}

In [103]:
f_name = "L8_Country"
scale = 30 ## usually 30 

In [104]:
# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': composite.toFloat(),
    'description': f_name,
    'folder':'GEE_Resources_Raw',   ## it will create a folder an dump the image there 
    'scale': scale,
    'crs': projection['crs'],
    'crsTransform': projection['transform'],
    'region': Japan.geometry(),
    'fileFormat': 'GeoTIFF',
    'maxPixels':1e13,
})
task.start()

while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(1200)
    
print(task.status())

Polling for task (id: S6MFACGLLLPV3BPB33PE6P2X).
Polling for task (id: S6MFACGLLLPV3BPB33PE6P2X).
Polling for task (id: S6MFACGLLLPV3BPB33PE6P2X).
Polling for task (id: S6MFACGLLLPV3BPB33PE6P2X).
Polling for task (id: S6MFACGLLLPV3BPB33PE6P2X).
Polling for task (id: S6MFACGLLLPV3BPB33PE6P2X).
{'state': 'COMPLETED', 'description': 'L8_Country', 'creation_timestamp_ms': 1651465066093, 'update_timestamp_ms': 1651471489590, 'start_timestamp_ms': 1651465082570, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/1IlG_KVAL6C3W9xrswbswFBRKjve2YXSu'], 'attempt': 1, 'id': 'S6MFACGLLLPV3BPB33PE6P2X', 'name': 'projects/earthengine-legacy/operations/S6MFACGLLLPV3BPB33PE6P2X'}
